### Import Require Libraries

In [60]:
import numpy as np
import sys

### Global Variables 

In [61]:
actions = {'U':(-1,0), 'D':(1,0),'R':(0,1),'L':(0,-1)}

### Creating the Maze class

In [62]:
class Maze():
    def __init__(self, input_maze):
        
#         self.maze = np.zeros((8,8))
#         self.maze[0, 0] = 2
#         self.maze[0, 1:6] = 1
#         self.maze[:4, 7] = 1
#         self.maze[2:6, 1] = 1
#         self.maze[2:6, 5] = 1
#         self.maze[7, :6] = 1
#         self.maze[5, 3:7] = 1
#         self.maze[4, 4:7] = 1
#         self.maze[3, 3] = 1
#         self.maze[3, 6] = 1

        self.maze = input_maze
        
        self.robot_position = (0,0)
        self.steps = 0
        
        self.allowed_states = None
        self.construct_allowed_states()
    
    def print_G_with_maze(self, g_dict):
        for row in range(len(self.maze)):
            for col in range(len(self.maze)):
                if self.maze[row][col]==0:
                    print('%.1f' % g_dict[(row,col)], end="\t")
                elif self.maze[row][col]==1:
                    print("X", end="\t")
                elif self.maze[row][col]==2:
                    print("R", end="\t")
            print() 
    
    def print_maze(self):
        print('The input maze:\n')
        for i in self.maze:
            for j in i:
                if j==0:
                    print("0",end="\t")
                elif j==1:
                    print("X", end="\t")
                elif j==2:
                    print("R", end="\t")
            print()
    
    def is_allowed_move(self, state, action):
        row, col = state
        updated_row = row + actions[action][0]
        updated_col = col + actions[action][1]
        
        maze_size = len(self.maze)
        
        if (0<=updated_row<maze_size and 0<=updated_col<maze_size) and (self.maze[updated_row][updated_col] in [0,2]):
            return True
       
        else:
            return False
    
    def construct_allowed_states(self):
        self.allowed_states = {}
        
        for row in range(len(self.maze)):
            for col in range(len(self.maze)):
                if self.maze[row][col] != 1:
                    self.allowed_states[(row,col)] = []
                    
                    for action in actions:
#                         print('({0},{1}) action:{2} => {3}'.format(row,col, action, self.is_allowed_move((row, col), action)))
                        if self.is_allowed_move((row, col), action):
                            self.allowed_states[(row,col)].append(action)
    
    # update maze and robot position 
    def update_maze(self, action): 
        row, col = self.robot_position
        self.maze[row][col] = 0
        
        row += actions[action][0]
        col += actions[action][1]
        
        self.robot_position = (row, col)
        self.maze[row][col] = 2
        self.steps += 1
    
    def is_game_over(self):
        if self.robot_position == (7,7):
            return True
        return False
    
    def give_reward(self):
        if self.robot_position == (7,7):
            return 0
        else:
            return -1
    
    def get_state_and_reward(self):
        return self.robot_position, self.give_reward()

### Creating Agent Class 

In [63]:
class Agent():
    def __init__(self, states, alpha=0.15, random_factor=0.2):
        self.state_history = [((0, 0), 0)] # state, reward
        self.alpha = alpha
        self.random_factor = random_factor
        
        # rewards table
        self.G = {}
        self.init_reward(states) # states is maze
    
    def init_reward(self, states):
        for row in range(len(states)):
            for col in range(len(states)):
#           for key in states.keys():
                self.G[(row,col)] = np.random.uniform(high=1.0, low=0.1)
#                 self.G[key] = np.random.uniform(high=1.0, low=0.1)
    
    def update_state_history(self, state, reward):
        self.state_history.append((state, reward))
    
    def learn(self):
        target = 0 # we know the "ideal" reward at (7,7)
        a = self.alpha
        
        for state, reward in reversed(self.state_history):
            self.G[state] = self.G[state] + a * (target - self.G[state])
            target += reward
            
        self.state_history = [] # reset the state_history
        self.random_factor -= 10e-5
    
    def choose_action(self, state, allowed_moves):
        next_move = None
        
        n = np.random.random()
        
        if n<self.random_factor:
            next_move = np.random.choice(allowed_moves)
        else:
            maxG = -10e15
            
            for action in allowed_moves:
#                     print([x for x in zip(state, actions[action])]) 
                new_state = tuple([sum(x) for x in zip(state, actions[action])]) # summation of two tupples
            
                if self.G[new_state] >= maxG:
                    next_move = action
                    maxG = self.G[new_state]
        
        return next_move

### Input Maze

In [70]:
create_maze = [
                [2, 1, 1, 1, 1, 1, 0, 1],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 1, 0, 0, 0, 1, 0, 1],
                [0, 1, 0, 1, 0, 1, 1, 1],
                [0, 1, 0, 0, 1, 1, 1, 0],
                [0, 1, 0, 1, 1, 1, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [1, 1, 1, 1, 1, 1, 0, 0]
              ]

### Craeting the maze and robot instance

In [71]:
maze = Maze(create_maze)
maze.print_maze()
# robot = Agent(maze.allowed_state, alpha=0.1, random_factor=0.25)
robot = Agent(maze.maze, alpha=0.1, random_factor=0.25)
# maze.allowed_states
# maze.construct_allowed_states()

The input maze:

R	X	X	X	X	X	0	X	
0	0	0	0	0	0	0	X	
0	X	0	0	0	X	0	X	
0	X	0	X	0	X	X	X	
0	X	0	0	X	X	X	0	
0	X	0	X	X	X	X	0	
0	0	0	0	0	0	0	0	
X	X	X	X	X	X	0	0	


###  Learning Proccess for the Robot

In [72]:
maze.print_maze()
print()

moveHistory = []

min_steps = float('inf') 

for i in range(5000):
    if i == 4999:
        print('Robot will always choose a step which reward is more close to zero.')
        maze.print_G_with_maze(robot.G)
        print()
        
    
    while not maze.is_game_over():
        state, reward = maze.get_state_and_reward()
        action = robot.choose_action(state, maze.allowed_states[state])
        maze.update_maze(action)
        state, reward = maze.get_state_and_reward()
        robot.update_state_history(state, reward)
        
        if maze.steps > 1000:
            maze.robot_position = (7,7)
        
    robot.learn()
    
    if min_steps > maze.steps:
        min_steps = maze.steps
        moveHistory = robot.state_history
    
    maze = Maze(create_maze)

The input maze:

R	X	X	X	X	X	0	X	
0	0	0	0	0	0	0	X	
0	X	0	0	0	X	0	X	
0	X	0	X	0	X	X	X	
0	X	0	0	X	X	X	0	
0	X	0	X	X	X	X	0	
0	0	0	0	0	0	0	0	
X	X	X	X	X	X	0	0	

Robot will always choose a step which reward is more close to zero.
-14.0	X	X	X	X	X	-602.6	X	
-12.0	-14.6	-27.7	-218.6	-426.0	-411.3	R	X	
-11.0	X	-40.5	-246.9	-467.5	X	-456.1	X	
-10.0	X	-114.4	X	-576.9	X	X	X	
-9.0	X	-65.1	-224.8	X	X	X	-16.9	
-8.0	X	-7.2	X	X	X	X	-1.4	
-7.0	-6.0	-5.0	-4.0	-3.0	-2.0	-1.0	-0.0	
X	X	X	X	X	X	-0.4	R	



### Final Choosen Path by The Robor 

In [74]:
print('Robot optimal movement:')

copy_maze = create_maze.copy()
copy_maze[0][0] = 'R'
for item in moveHistory:
    
    copy_maze[item[0][0]][item[0][1]] = 'R'    

for i in range(len(copy_maze)):
    for j in range(len(copy_maze)):
        print(copy_maze[i][j], end='\t')
    print()

Robot optimal movement:
R	1	1	1	1	1	0	1	
R	0	0	0	0	0	2	1	
R	1	0	0	0	1	0	1	
R	1	0	1	0	1	1	1	
R	1	0	0	1	1	1	0	
R	1	0	1	1	1	1	0	
R	R	R	R	R	R	R	0	
1	1	1	1	1	1	R	R	
